In [5]:
#pip install shap

In [6]:
#importing packages
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests                                      # reading data
from io import StringIO

from sklearn.datasets import fetch_openml            # common data set access
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from sklearn.metrics import zero_one_loss as J01

import sklearn.tree as tree

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)  

In [7]:
# Reading in the CSV file
dia_data = pd.read_csv("data/diabetic_data.csv")

#print(dia_X.shape)
# View the first 5 rows
print(dia_data.head())
#print(f"NumFeatures: {len(dia_data.columns)} \nFeatures: {dia_data.columns}")
#print(f"Summary: {dia_data.info}")

   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No                   No

In [8]:
#data = dia_data.to_numpy()
X, Y = dia_data.iloc[:,:-1], dia_data.iloc[:,-1]
#train 70% temp 30%
Xtr, Xtemp, Ytr, Ytemp = train_test_split(X, Y, test_size=0.3, random_state=seed)


# validation: 10% Test: 20%
Xva, Xtest, Yva, Ytest = train_test_split(Xtemp, Ytemp, test_size=0.67, random_state=seed)

print(Xtr.shape, Ytr.shape)
print(Xva.shape, Yva.shape)
print(Xtest.shape, Ytest.shape)

(71236, 49) (71236,)
(10074, 49) (10074,)
(20456, 49) (20456,)


# Priya's code


In [9]:
Xtr_priya=Xtr.iloc[:,33:50]
Xtr_priya

,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
47866,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes
24673,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
61033,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes
74444,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No
56971,Steady,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89460,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes
60620,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes
34086,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes
58067,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No


## Description of the columns

* These columns are different diabetes medicines prescribed
* up indicates that dosage was increased
* sown if the dosage was decreased
* steady if the dosage remained the same
* No if it wasnt prescribed
 


In [10]:
Xtr_priya.describe()

,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
count,71236,71236,71236,71236,71236,71236,71236,71236,71236,71236,71236,71236,71236,71236,71236,71236
unique,4,4,4,4,2,3,1,1,4,4,2,2,2,2,2,2
top,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes
freq,66072,66775,71014,71209,71235,71213,71236,71236,33094,70734,71228,71235,71235,71235,38200,54798


In [21]:
#Printing the values in each column and their frequency

for i in range(len(Xtr_priya.columns)):
    print(Xtr_priya.iloc[:,i].value_counts())
    print()

pioglitazone
No        66072
Steady     4917
Up          168
Down         79
Name: count, dtype: int64

rosiglitazone
No        66775
Steady     4273
Up          122
Down         66
Name: count, dtype: int64

acarbose
No        71014
Steady      215
Up            6
Down          1
Name: count, dtype: int64

miglitol
No        71209
Steady       22
Down          4
Up            1
Name: count, dtype: int64

troglitazone
No        71235
Steady        1
Name: count, dtype: int64

tolazamide
No        71213
Steady       22
Up            1
Name: count, dtype: int64

examide
No    71236
Name: count, dtype: int64

citoglipton
No    71236
Name: count, dtype: int64

insulin
No        33094
Steady    21587
Down       8563
Up         7992
Name: count, dtype: int64

glyburide-metformin
No        70734
Steady      493
Up            5
Down          4
Name: count, dtype: int64

glipizide-metformin
No        71228
Steady        8
Name: count, dtype: int64

glimepiride-pioglitazone
No        71235
Stead

Observations:
* examide and citoglipton have only one value and its "No". Can be dropped if necessary. Keeping it for now. 


## One hot Encoding variables

In [27]:
from sklearn.preprocessing import OneHotEncoder
encoder3 = OneHotEncoder(handle_unknown='error', sparse_output=False)
Xtr_priya_encoded = encoder3.fit_transform(Xtr_priya)

In [28]:
new_cols = encoder3.get_feature_names_out(Xtr_priya.columns)
Xtr_priya_encoded = pd.DataFrame(Xtr_priya_encoded, columns= new_cols)

In [29]:
Xtr_priya_encoded

,pioglitazone_Down,pioglitazone_No,pioglitazone_Steady,pioglitazone_Up,rosiglitazone_Down,rosiglitazone_No,rosiglitazone_Steady,rosiglitazone_Up,acarbose_Down,acarbose_No,...,glimepiride-pioglitazone_No,glimepiride-pioglitazone_Steady,metformin-rosiglitazone_No,metformin-rosiglitazone_Steady,metformin-pioglitazone_No,metformin-pioglitazone_Steady,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes
0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
71232,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
71233,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
71234,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
